In [219]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import gc
import cv2
from tqdm import tqdm
# from tensorflow import keras
# from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
# from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

from matplotlib import pyplot as plt


notebook_dir = os.getcwd()
target_path = os.path.abspath(os.path.join(notebook_dir, '../Python_Code'))
sys.path.append(target_path)
from dataGenerator import DataGenerator

import util

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [337]:
csvs = pd.read_csv('../EDA/SAIDAS/04-csv_summary.csv', usecols=['test', 'environment', 'monitor', 'slot', 'csv_file_path'])
csvs.drop_duplicates(inplace=True)
csvs = csvs.loc[(csvs['test']=='proximity')&(csvs['slot'].str.contains("Test"))]
csvs.reset_index(drop=True, inplace=True)

In [ ]:
# csvs = csvs.iloc[0:1]

In [338]:
def processing(row):
  start_time = time.time()
  
  batchsize = 48

  # Constants
  window_size = 50
  NoOfSubcarrier = 242
  labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']
  labels_array = np.array(labels)

  # File and directory paths
  environment, csv_file_path, monitor, slot = row[['environment', 'csv_file_path', 'monitor', 'slot']]
  target_subject = slot.replace('Test_', '')
  test_dir = os.path.dirname(csv_file_path)
  model_dir = os.path.dirname(os.path.dirname(os.path.dirname(csv_file_path)))
  gradcam_dir = test_dir.replace('Data', 'GradCam')
  model_path = f'{model_dir}/{environment}_{monitor}_{target_subject}.h5'
  csv_output_file = csv_file_path.replace('Data', 'GradCam')
  gradcam_output_file = csv_output_file.replace('.csv', '.npy')

  test_gen = DataGenerator(test_dir, csv_file_path, NoOfSubcarrier, len(labels), (window_size, NoOfSubcarrier, 2), batchsize=batchsize, shuffle=False)

  model = load_model(model_path)

  if not os.path.exists(gradcam_output_file):
    gradcam_basefolder = os.path.dirname(gradcam_output_file)
    if not os.path.exists(gradcam_basefolder):
      os.makedirs(gradcam_basefolder)

    heatmaps = None
    
    csv = pd.read_csv(csv_file_path)
    csv['predicted_label'] = None
    csv['predicted_prob'] = None

    for batch_index in tqdm(range(len(test_gen))):
      X, Y = test_gen[batch_index]
      heatmap, top_pred_index, top_class_channel = util.batch_make_gradcam_heatmap(X, model)

      if heatmaps is None:
        heatmaps = heatmap
      else:
        heatmaps = np.concatenate((heatmaps, heatmap), axis=0)

      batch_start_index = batch_index * batchsize
      batch_end_index = batch_start_index + len(X)
      csv.loc[batch_start_index:batch_end_index - 1, 'predicted_label'] = labels_array[top_pred_index.numpy()]
      csv.loc[batch_start_index:batch_end_index - 1, 'predicted_prob'] = top_class_channel.numpy()


    sample_index_start = len(test_gen) * batchsize
    test_gen = DataGenerator(test_dir, csv_file_path, NoOfSubcarrier, len(labels), (window_size, NoOfSubcarrier, 2), batchsize=1, shuffle=False)

    for sample_index in tqdm(range(sample_index_start, len(test_gen))):
      X, Y = test_gen[sample_index]
      heatmap, top_pred_index, top_class_channel = util.batch_make_gradcam_heatmap(X, model)

      heatmaps = np.concatenate((heatmaps, heatmap), axis=0)

      csv.at[sample_index, 'predicted_label'] = labels_array[top_pred_index.numpy()].item()
      csv.at[sample_index, 'predicted_prob'] = top_class_channel.numpy()[0]

    csv['right'] = csv['predicted_label'] == csv['label']
    csv.to_csv(csv_output_file, index=False)
    np.save(gradcam_output_file, heatmaps)

    # Free GPU memory
    del model
    gc.collect()
    tf.keras.backend.clear_session()

    end_time = time.time()
    elapsed_time = end_time - start_time
    row['elapsed_time_seconds'] = elapsed_time

  return row  

csvs = csvs.apply(processing, axis=1)
csvs


100%|██████████| 32/32 [00:02<00:00, 12.87it/s]


,csv_file_path,elapsed_time_seconds,environment,monitor,slot,test
0,../Data/proximity/Classroom/80MHz/3mo/m1/Slots...,NaN,Classroom,m1,Test_m1,proximity
1,../Data/proximity/Classroom/80MHz/3mo/m1/Slots...,204.341243,Classroom,m1,Test_m2,proximity
2,../Data/proximity/Classroom/80MHz/3mo/m1/Slots...,202.726163,Classroom,m1,Test_m3,proximity
3,../Data/proximity/Classroom/80MHz/3mo/m2/Slots...,211.516498,Classroom,m2,Test_m1,proximity
4,../Data/proximity/Classroom/80MHz/3mo/m2/Slots...,206.339200,Classroom,m2,Test_m2,proximity
5,../Data/proximity/Classroom/80MHz/3mo/m2/Slots...,201.342412,Classroom,m2,Test_m3,proximity
6,../Data/proximity/Classroom/80MHz/3mo/m3/Slots...,223.379641,Classroom,m3,Test_m1,proximity
7,../Data/proximity/Classroom/80MHz/3mo/m3/Slots...,207.715699,Classroom,m3,Test_m2,proximity
8,../Data/proximity/Classroom/80MHz/3mo/m3/Slots...,212.891093,Classroom,m3,Test_m3,proximity
9,../Data/proximity/Office/80MHz/3mo/m1/Slots/Te...,355.288625,Office,m1,Test_m1,proximity
